In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8051"

In [4]:
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

def research_messages_hook(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

research_message_listener = {
    "handler": research_messages_hook,
    "topic": "basicmessages"
}



In [5]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [6]:
agent_controller.register_listeners([research_message_listener], defaults=True)

In [7]:
#Paste in invitation from data_owner agent
invite = {'connection_id': '6ceb16b7-c95b-4f2f-9ee6-7f83232807e5', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '815e1e53-37d0-4aa9-b2a3-4683b6f708f6', 'label': 'Bob', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['2TMwBDhLQU6ef1x4AaFdL58b9PiRbukVBnYstakWuNUV']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiODE1ZTFlNTMtMzdkMC00YWE5LWIyYTMtNDY4M2I2ZjcwOGY2IiwgImxhYmVsIjogIkJvYiIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJyZWNpcGllbnRLZXlzIjogWyIyVE13QkRoTFFVNmVmMXg0QWFGZEw1OGI5UGlSYnVrVkJuWXN0YWtXdU5VViJdfQ=='}



In [8]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)
# Print out accepted Invite and Researcher ID
print("Connection", response)
researcher_id = response["connection_id"]


ACCEPTED CONNECTION -  {'updated_at': '2020-06-25 15:55:54.621110Z', 'created_at': '2020-06-25 15:55:54.601676Z', 'my_did': 'YZjeSRBv5WJSvwtqATeAUJ', 'their_label': 'Bob', 'invitation_key': '2TMwBDhLQU6ef1x4AaFdL58b9PiRbukVBnYstakWuNUV', 'routing_state': 'none', 'invitation_mode': 'once', 'accept': 'manual', 'connection_id': '1d63886a-85dc-48e1-89bd-3109bf665fe0', 'state': 'request', 'initiator': 'external', 'request_id': '1f21520e-adf7-4b21-a24a-e35058cbf6b0'}
{'updated_at': '2020-06-25 15:55:54.621110Z', 'created_at': '2020-06-25 15:55:54.601676Z', 'my_did': 'YZjeSRBv5WJSvwtqATeAUJ', 'their_label': 'Bob', 'invitation_key': '2TMwBDhLQU6ef1x4AaFdL58b9PiRbukVBnYstakWuNUV', 'routing_state': 'none', 'invitation_mode': 'once', 'accept': 'manual', 'connection_id': '1d63886a-85dc-48e1-89bd-3109bf665fe0', 'state': 'request', 'initiator': 'external', 'request_id': '1f21520e-adf7-4b21-a24a-e35058cbf6b0'}
Connection {'updated_at': '2020-06-25 15:55:54.621110Z', 'created_at': '2020-06-25 15:55:54

In [9]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA
{'thread_id': 'e4d378ee-2867-4e4a-b674-64c7b6f2a2d0'}


In [10]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

RESEARCH AGENT CONNECTION
{'updated_at': '2020-06-25 15:56:44.224945Z', 'created_at': '2020-06-25 15:55:54.601676Z', 'my_did': 'YZjeSRBv5WJSvwtqATeAUJ', 'their_label': 'Bob', 'invitation_key': '2TMwBDhLQU6ef1x4AaFdL58b9PiRbukVBnYstakWuNUV', 'routing_state': 'none', 'invitation_mode': 'once', 'accept': 'manual', 'their_did': 'CGpVrCGcbpPSmTX5WiPB9F', 'connection_id': '1d63886a-85dc-48e1-89bd-3109bf665fe0', 'state': 'active', 'initiator': 'external', 'request_id': '1f21520e-adf7-4b21-a24a-e35058cbf6b0'}


In [12]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)

Handle research messages  {'connection_id': '1d63886a-85dc-48e1-89bd-3109bf665fe0', 'message_id': 'f48abbbd-5daa-419f-9b55-21b5264319e9', 'content': 'hello from data owner world! WOOPWOOP', 'state': 'received'} 1d63886a-85dc-48e1-89bd-3109bf665fe0
Handle research messages  {'connection_id': '1d63886a-85dc-48e1-89bd-3109bf665fe0', 'message_id': '0fe8bab8-d0dc-4bbd-8b7f-31ddf84b4d83', 'content': 'THIS IS MY RESPONSE - DATA AGENT', 'state': 'received'} 1d63886a-85dc-48e1-89bd-3109bf665fe0
BASIC MESSAGE - RESEARCH -> DATA
{}
